# VacationPy

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_2 import g_key


### Store Part I results into DataFrame

- Load the csv exported in Part I to a DataFrame

In [9]:
# CSV file with data
weather_metadata = "output_data/my_weather_data.csv"

# Read in our weather data and the results 
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single df
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,Country,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,GL,Sisimiut,66.9395,-53.6735,31.71,92,96,2.26,1601424000
1,1,AU,Hobart,-42.8794,147.3294,59.31,63,95,9.33,1601424000
2,2,NZ,Bluff,-46.6000,168.3333,45.32,90,98,17.47,1601424000
3,3,AR,Mar del Plata,-38.0023,-57.5575,64.51,58,24,13.96,1601424000
4,4,ZA,East London,-33.0153,27.9116,59.56,66,96,0.09,1601424000


### Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.

In [10]:
# Configure gmaps with API key

gmaps.configure(api_key=g_key)

In [11]:
# Storing lat and lng into  locations (float)
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

# measuring humidity
hum_rate = weatherdf["Humidity"].astype(float)

In [13]:
# Create a Heatmap layer
# jupyter nbextension enable --py gmaps (if maps aren't showing)

heat_fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

# formating syntax for our heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

heat_fig.add_layer(heat_layer)

#plt.savefig("vacation_heatmap.png") 
heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [15]:
# Max temp lower than 80 degrees but higher than 70. Use .loc
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

# Wind speed less than 10
third = second.loc[second["Wind Speed"]<10,:]
third

# Zero cloudiness [==0]
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,Country,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
35,35,BR,Caravelas,-17.7125,-39.2481,74.79,83,0,7.78,1601424000
50,50,IR,Iranshahr,27.2025,60.6848,76.44,24,0,3.62,1601424000
101,101,BR,Belmonte,-15.8631,-38.8828,75.65,80,0,8.37,1601424000
172,172,MZ,Angoche,-16.2325,39.9086,74.19,89,0,7.20,1601424000
303,303,TN,Tataouine,32.9297,10.4518,72.99,75,0,5.53,1601424000


### Hotel Map

- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [16]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

# New df
# hotel_df['column name'] = fith["name"]
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']

# Print
hotel_df.head()

,City,Country,Lat,Lng
35,Caravelas,BR,-17.7125,-39.2481
50,Iranshahr,IR,27.2025,60.6848
101,Belmonte,BR,-15.8631,-38.8828
172,Angoche,MZ,-16.2325,39.9086
303,Tataouine,TN,32.9297,10.4518


In [ ]:
#https://maps.googleapis.com/maps/api/place/textsearch/json?location=

In [17]:
# create a loop using iterrows
for index, row in hotel_df.iterrows():

    # Get hotel type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # Assemble url and make API request (Be carefull making too many API requests)
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # response/results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 35: Caravelas.
Closest hotel to Caravelas at -17.7125 , -39.2481 is Mateus' S/A.
------------
Retrieving Results for Index 50: Iranshahr.
Closest hotel to Iranshahr at 27.2025 , 60.6848 is Farzad Cybercafe.
------------
Retrieving Results for Index 101: Belmonte.
Closest hotel to Belmonte at -15.8631 , -38.8828 is Expresso Brasileiro.
------------
Retrieving Results for Index 172: Angoche.
Closest hotel to Angoche at -16.2325 , 39.9086 is Boy's guest house.
------------
Retrieving Results for Index 303: Tataouine.
Closest hotel to Tataouine at 32.9297 , 10.4518 is Page USTataouine Europe.
------------
Retrieving Results for Index 416: Manbij.
Closest hotel to Manbij at 36.5281 , 37.9549 is School October 16.
------------
Retrieving Results for Index 420: Aligarh.
Closest hotel to Aligarh at 27.8833 , 78.0833 is Bishen Skin Centre.
------------
Retrieving Results for Index 478: Hūn.
Closest hotel to Hūn at 29.1268 , 15.9477 is الوطن موبايل.
------------
Retr

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure